### Build a Retrieval Augmented Generation (RAG)

In [ ]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install groq  -q
!pip install qdrant-client -q
!pip install sentence-transformers -q
!pip install langchain -q
!pip install pypdf  -q 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [76]:
import os
from groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

In [ ]:
load_dotenv('.env')  
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [78]:
# Initialize embedding model using Sentence Transformers
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_size = embedding_model.embed_query("test").__len__()

In [79]:
# Initialize Qdrant Client (using in-memory or adjust to your needs)
qdrant = QdrantClient(":memory:")

In [80]:
# Create a Qdrant collection to hold document vectors
qdrant.recreate_collection(
    collection_name="documents",
    vectors_config={"size": vector_size, "distance": "Cosine"}
)

True

In [81]:
# Load PDF document
loader = PyPDFLoader("../pdf/ข้อมูลยา 50 ชนิด.pdf")
documents = loader.load()

In [82]:
# Split the documents into smaller chunks
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
document_chunks = splitter.split_documents(documents)

In [83]:
# Extract text from document chunks
texts = [doc.page_content for doc in document_chunks]

# Convert texts into vectors
vectors = embedding_model.embed_documents(texts)

# Upsert data into Qdrant
points = [PointStruct(id=i, vector=vectors[i], payload={"text": texts[i]}) for i in range(len(texts))]
qdrant.upsert(collection_name="documents", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def search_documents(query):
    # Convert query to vector
    query_vector = embedding_model.embed_query(query)
    
    # Search Qdrant for similar documents
    search_results = qdrant.search(
        collection_name="documents",
        query_vector=query_vector,
        limit=4  # Retrieve top 5 relevant documents
    )
    
    # Check if results are found
    if not search_results:
        return []  # Return empty if no documents found
    
    # Extract text from results 
    return [hit.payload.get("text", "เอกสารไม่มีข้อความ") for hit in search_results]

In [85]:
def generate_answer(query):
    '''
        
    '''
    # Retrieve relevant documents from Qdrant
    retrieved_docs = search_documents(query)

    # Check if any documents were retrieved
    if not retrieved_docs:
        return "ไม่พบข้อมูลที่เกี่ยวข้อง"  # "No relevant information found"

    # Create the context for the language model
    context = "\n".join([str(doc) for doc in retrieved_docs if isinstance(doc, str)])

    # Check if context has content
    if not context.strip():
        # "No relevant information found"
        return "ไม่พบข้อมูลที่เกี่ยวข้อง"

    prompt = f"ข้อมูลอ้างอิง:\n{context}\n\nคำถาม: {query}\n\nคำตอบ:"

    # Initialize Groq API client
    groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

    try:
        # Requesting completion from Groq API
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant", messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"เกิดข้อผิดพลาดในการสร้างคำตอบ: {str(e)}"
    # "Error occurred in generating the answer"

In [86]:
# 🔥 Test the question!
query = "มีรายการอะไรบ้าง"  
answer = generate_answer(query)
print(answer)

มีรายการยาทั้งหมด 25 รายการ


In [87]:
# 🔥 Test the question!
query = "ขอรายอะเอียดของยา  พาราเซตามอล(Paracetamol)"  
answer = generate_answer(query)
print(answer)

ยาที่คุณถามถึงคือ "พาราเซตามอล" (Paracetamol) และมีรายละเอียดดังนี้

- ส่วนผสม: พาราเซตามอล
- สูตรสำหรับสูงสภาวะ: พาราเซตามอล-สัมพันธุ์: บรรเทาอาการปวดและอาการหนาว
- ขั้นตอนการใช้:
 + ผู้ใหญ่: ผู้ใหญ่ให้ประเมินที่ 0.5-1 กรัม (5-10 ม.ลล.) ทุกๆ 4-6 ชั่วโมงมากสุด 4-6 ชั่วโมงต่อวัน
 + แต่ผู้บรอกดตามข้อมูลอย่างละเอียดและไม่เกิน 4 กรัมต่อวัน
 + นอกจากนี้ยังมีขอยกเว้นสำหรับเด็ก เพราะไม่เกิดผลมากนัก และการตั้งแต่วันแรกสามารถใช้ได้แต่ลดน้อยลงเหลือ 1 ชั่วโมง
- ค่าแนะนำ: บรรเทาอาการปวดและอาการหนาว
- ขั้นตอนใช้สำหรับแก้ไขปัญหา:
 + อาจใช้เป็นภาชนะเพื่อเจ็บปวด
 * (ดูได้จากข้อความ "ห้ามใช้สำหรับแก้ไขทันทีต่อเวลาจากการเสีย"
 - คำเตือน: เมื่อใช้ยาวนานหรือจำนวนมาก ได้เจ็บมากและเสียระบบหรือปิติ
